In [1]:
! pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 53.8 MB/s eta 0:00:00


In [5]:
import pymupdf as fitz

def extract_text_with_formatting(pdf_path, skip_pages=10):
    doc = fitz.open(pdf_path)
    text_data = []

    def int_to_rgb(color_int, colorspace):
        """Wandle den Farb-Integer in RGB-Tupel um, abhängig vom Farbraum."""
        if colorspace == 0:  # DeviceGray (Graustufen)
            gray = color_int & 255
            return (gray, gray, gray)
        elif colorspace == 1:  # DeviceRGB (Standard-RGB)
            return ((color_int >> 16) & 255, (color_int >> 8) & 255, color_int & 255)
        elif colorspace == 2:  # DeviceCMYK (Annäherung, da PyMuPDF keine direkte Umwandlung macht)
            c = (color_int >> 24) & 255
            m = (color_int >> 16) & 255
            y = (color_int >> 8) & 255
            k = color_int & 255
            # Annäherung: Konvertiere CMYK grob zu RGB
            r = 255 * (1 - c / 255) * (1 - k / 255)
            g = 255 * (1 - m / 255) * (1 - k / 255)
            b = 255 * (1 - y / 255) * (1 - k / 255)
            return (int(r), int(g), int(b))
        else:
            return (0, 0, 0)  # Standard: Schwarz

    # Schleife über Seiten
    for page_num in range(skip_pages, 176):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]  # Extrahiert Textblöcke

        # Schleife über Textblöcke
        for block in blocks:
            if block["type"] == 0:  # Textblock
                for line in block["lines"]:
                    for span in line["spans"]:
                        # Extrahiere Farbraum (Standard: RGB = 1)
                        colorspace = span.get("colorspace", 1)
                        color_rgb = int_to_rgb(span["color"], colorspace)

                        # Speichere Text + Schriftgröße + Fett + RGB-Farbe
                        text_data.append({
                            "text": span["text"],
                            "size": span["size"],  # Schriftgröße
                            "bold": span["flags"] & 2 != 0,  # Prüft, ob der Text fett ist
                            "color_rgb": color_rgb,  # RGB-Tupel
                            "colorspace": colorspace  # Farbraum zur Kontrolle
                        })

    doc.close()
    return text_data

# PDF-Dateipfad
pdf_path = "/content/Wahlprogramm_AFD.pdf"

# Text mit Formatierungsinformationen extrahieren
text_data = extract_text_with_formatting(pdf_path)

# Beispiel: Zeige die ersten 130 Textabschnitte mit Formatierung
for item in text_data[:130]:
    print(f"Text: {item['text']}, Size: {item['size']}, Bold: {item['bold']}, Color: {item['color_rgb']}, Colorspace: {item['colorspace']}")


Text: 10, Size: 10.0, Bold: False, Color: (0, 0, 0), Colorspace: 1
Text: SOZIALE MARKTWIRTSCHAFT, Size: 14.0, Bold: True, Color: (0, 0, 0), Colorspace: 1
Text: Wir stehen fest zu den Prinzipien der Sozialen Marktwirtschaft, die , Size: 9.0, Bold: False, Color: (0, 0, 0), Colorspace: 1
Text: seit Jahrzehnten Wohlstand und sozialen Frieden in unserem Land , Size: 9.0, Bold: False, Color: (0, 0, 0), Colorspace: 1
Text: sichern. , Size: 9.0, Bold: False, Color: (0, 0, 0), Colorspace: 1
Text: Wir wollen in einer Zeit globaler Herausforderungen die Grundwerte , Size: 9.0, Bold: False, Color: (0, 0, 0), Colorspace: 1
Text: unserer Wirtschaftsordnung bewahren und so weiterentwickeln, , Size: 9.0, Bold: False, Color: (0, 0, 0), Colorspace: 1
Text: dass wir eine freiheitliche und erfolgreiche Gesellschaft gewährleis-, Size: 9.0, Bold: False, Color: (0, 0, 0), Colorspace: 1
Text: ten, in der die Bürger ihr Leben eigenverantwortlich und selbstbe-, Size: 9.0, Bold: False, Color: (0, 0, 0), Colorspa

In [6]:
## Output anschauen: Normale Schrift ist in Schriftgröße 9, Überschriften sind größer und nur Großbuchstaben
## Seitenzahlen stehen in einzelnen Zeilen in Größe 10
## viele leere Zeilen


import pandas as pd

def split_sections(text_data, min_size=9, bold_threshold=True):
    sections = []
    current_section = []

    for item in text_data:
        text = item["text"]

        if item["size"] > min_size and text.isupper() and item['bold'] == bold_threshold: # Überschriften überspringen + Abschnitt definieren
            if current_section:
                sections.append(" ".join(current_section).strip())
            current_section = []  # Abschnitt zurücksetzen

        elif text.isdigit() and len(text) <= 4:  # Seitenzahlen überspringen
            continue

        elif text.isupper() and item["bold"] == False: # Kopfzeile überspringen überspringen
            continue
        else:
            current_section.append(text)

    if current_section:
        sections.append(" ".join(current_section).strip()) # Text hinzufügen

    return sections



# Abschnitte extrahieren
sections = split_sections(text_data)

# Erstelle eine Tabelle mit den Abschnitten
df = pd.DataFrame(sections, columns=["Abschnitt"])

print(df[:10])

                                           Abschnitt
0  Wir stehen fest zu den Prinzipien der Sozialen...
1  Wir streben eine Rentenversicherung an, mit de...
2  Einen Anspruch auf Leistungen aus der Arbeitsl...
3  In Deutschland sind die Beiträge für die Krank...
4  Wir streben einen wirtschaftlichen Zustand an,...
5  Da alle erwachsenen Bürgergeldempfänger grunds...
6  Beim bestehenden deutschen Gesundheitssystem z...
7  In weiten Teilen Deutschlands herrscht unter a...
8  Eine Infrastruktur für Deutschlands Zukunft Ni...
9  Die jahrzehntelange Fokussierung auf die ideol...


In [ ]:
import pandas as pd

def split_sections(text_data, min_size=9, bold_threshold=True):
    sections = []
    current_section = []

    # Hilfsfunktion zur Erkennung von Überschriften, die zwischen zwei leeren Zeilen stehen
    def is_spacing_heading(index):
        # Prüfen, ob vorherige und nächste Zeile leer sind (falls vorhanden)
        prev_empty = index > 0 and text_data[index - 1]["text"].strip() == ""
        next_empty = index + 1 < len(text_data) and text_data[index + 1]["text"].strip() == ""
        return prev_empty and next_empty

    for i, item in enumerate(text_data):
        text = item["text"].strip()

        # Seitenzahlen überspringen
        if text.isdigit() and len(text) <= 4:
            continue

        # Leere Zeile überspringen
        if text == "":
            continue

        # Hauptüberschrift: groß, fett, Größe > min_size
        if item["size"] > min_size and text.isupper() and item['bold'] == bold_threshold:
            if current_section:
                sections.append(" ".join(current_section).strip())
            current_section = []
            continue

        # Zwischenüberschrift: groß, nicht fett
        elif text.isupper() and not item["bold"]:
            continue

        # Neue Logik: Überschrift zwischen zwei leeren Zeilen
        elif is_spacing_heading(i):
            if current_section:
                sections.append(" ".join(current_section).strip())
            current_section = []
            continue

        # Standard-Text zum aktuellen Abschnitt hinzufügen
        else:
            current_section.append(text)

    # Letzten Abschnitt hinzufügen
    if current_section:
        sections.append(" ".join(current_section).strip())

    return sections


# Test der Funktion
sections = split_sections(text_data)

# Erstelle DataFrame
df = pd.DataFrame(sections, columns=["Abschnitt"])
print(df)


                                            Abschnitt
0   Wir stehen fest zu den Prinzipien der Sozialen...
1   Wir streben eine Rentenversicherung an, mit de...
2   Einen Anspruch auf Leistungen aus der Arbeitsl...
3   In Deutschland sind die Beiträge für die Krank...
4   Wir streben einen wirtschaftlichen Zustand an,...
5   Da alle erwachsenen Bürgergeldempfänger grunds...
6   Beim bestehenden deutschen Gesundheitssystem z...
7   In weiten Teilen Deutschlands herrscht unter a...
8   Eine Infrastruktur für Deutschlands Zukunft Ni...
9   Die jahrzehntelange Fokussierung auf die ideol...
10  Verkehrspolitik – ideologiefrei und bedarfsger...
11  Die Digitalisierung ist aus der modernen Gesel...
12  Der Staat erhebt Steuern auch mit dem Ziel, du...
13                                                  -
14  Deutschland hat kein Einnahmen-, sondern ein A...
15  Das deutsche Steuerrecht ist mit über 30.000 V...
16  Zurück zu Recht, Wohlstand und sozialem Friede...
17  Reine Luft und sauberes 

In [7]:
# Speichern der Tabelle als CSV-Datei
df.to_csv("/content/afd.csv", index=False, encoding="utf-8")